### Import necessary tools

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from cma.bbobbenchmarks import instantiate
from leap_ec.problem import FunctionProblem
import numpy as np

In [3]:
from pyhms.config import TreeConfig, CMALevelConfig, EALevelConfig, DELevelConfig
from pyhms.tree import DemeTree
from pyhms.demes.single_pop_eas.sea import SEA
from pyhms.sprout.sprout_mechanisms import SproutMechanism
from pyhms.sprout.sprout_filters import NBC_FarEnough, DemeLimit, LevelLimit
from pyhms.sprout.sprout_generators import NBC_Generator
from pyhms.stop_conditions.usc import dont_stop, metaepoch_limit

### Prepare function instances
Diverse test set with focus on multimodal problems. 2 dimensional for visualization.

In [4]:
separable_rastrigin = instantiate(3, 1)
attractive_sector = instantiate(6, 1)
step_elipsoid = instantiate(7, 1)
bent_cigar = instantiate(12, 1)
asymetric_rastrigin = instantiate(15, 1)
griewank_rosenbrock = instantiate(19, 1)
katsuura = instantiate(23, 1)
lunacek_rastrigin = instantiate(24, 1)

testbed ={"separable_rastrigin(3)": separable_rastrigin,
          "attractive_sector(6)": attractive_sector,
          "step_elipsoid(7)": step_elipsoid,
          "bent_cigar(12)": bent_cigar,
          "asymetric_rastrigin(15)": asymetric_rastrigin,
          "griewank_rosenbrock(19)": griewank_rosenbrock,
          "katsuura(23)": katsuura,
          "lunacek_rastrigin(24)": lunacek_rastrigin
          }

### Algorithm setup

In [9]:
bounds_base = [(-5.0, 5.0)]
gsc = metaepoch_limit(35)
lvl_2_lsc = dont_stop()
lvl_1_generations = 1
lvl_2_generations = 8
pop_size = 40
mutation_std = 1.0
sigma0 = 2.0

dimensionality = 10

sprout_cond = SproutMechanism(NBC_Generator(2.0, 0.4), [NBC_FarEnough(2.0, 2), DemeLimit(1)], [LevelLimit(6)])

In [7]:
def setup_hms_for_bbob(function_problem, bounds):

    config = [
    EALevelConfig(
        ea_class=SEA, 
        generations=lvl_1_generations, 
        problem=function_problem, 
        bounds=bounds, 
        pop_size=pop_size,
        mutation_std=mutation_std,
        lsc=dont_stop()
        ),
    CMALevelConfig(
        generations=lvl_2_generations,
        problem=function_problem, 
        bounds=bounds,
        sigma0=sigma0,
        lsc=lvl_2_lsc
        )
    ]

    config = TreeConfig(config, gsc, sprout_cond)
    return DemeTree(config)

### Benchmark
Initial quality evaluation on budget similar to GECCO paper.

In [8]:
for name, problem in testbed.items():
    hms_tree = setup_hms_for_bbob(FunctionProblem(problem[0], maximize=False), bounds_base*dimensionality)
    print(f"Running {name}")
    while not hms_tree._gsc(hms_tree):
        hms_tree.metaepoch_count += 1
        hms_tree.run_metaepoch()
        if not hms_tree._gsc(hms_tree):
            hms_tree.run_sprout()

    f = lambda x: lvl_1_generations if x == 1 else lvl_2_generations
    print(f"Best fitness: {np.max(hms_tree.optima).fitness - problem[1]}")
    print(f"Numer of demes: {len(hms_tree.all_demes)}")
    print(f"Evaluation budget: {np.sum([np.sum([len(pop) for pop in deme.history])*(f(level)) for level, deme in hms_tree.all_demes])}")
    print('-')

Running separable_rastrigin(3)
Best fitness: 4.974926600199808
Numer of demes: 2
Evaluation budget: 14750
-
Running attractive_sector(6)
Best fitness: 4.869440731880559e-05
Numer of demes: 6
Evaluation budget: 15650
-
Running step_elipsoid(7)
Best fitness: 0.03949616931494404
Numer of demes: 6
Evaluation budget: 15110
-
Running bent_cigar(12)
Best fitness: 0.24381627095863223
Numer of demes: 3
Evaluation budget: 15070
-
Running asymetric_rastrigin(15)
Best fitness: 12.934457746102566
Numer of demes: 2
Evaluation budget: 14750
-
Running griewank_rosenbrock(19)
Best fitness: 5.30956185902933
Numer of demes: 4
Evaluation budget: 15350
-
Running katsuura(23)
Best fitness: 6.076413502537769
Numer of demes: 5
Evaluation budget: 15530
-
Running lunacek_rastrigin(24)
Best fitness: 79.67485587562335
Numer of demes: 3
Evaluation budget: 15090
-


In [13]:
for name, problem in testbed.items():
    print(f"Running {name}")
    f = lambda x: lvl_1_generations if x == 1 else lvl_2_generations
    bests = []
    demes = []
    evaluations = []
    for _ in range(50):
        hms_tree = setup_hms_for_bbob(FunctionProblem(problem[0], maximize=False), bounds_base*dimensionality)
        while not hms_tree._gsc(hms_tree):
            hms_tree.metaepoch_count += 1
            hms_tree.run_metaepoch()
            if not hms_tree._gsc(hms_tree):
                hms_tree.run_sprout()
        bests += [np.max(hms_tree.optima).fitness - problem[1]]
        demes.append(len(hms_tree.all_demes))
        evaluations.append(np.sum([np.sum([len(pop) for pop in deme.history])*(f(level)) for level, deme in hms_tree.all_demes]))

    print(f"Best fitness: {np.mean(bests)}")
    print(f"Numer of demes: {np.mean(demes)}")
    print(f"Evaluation budget: {np.mean(evaluations)}")
    print('-')

Running separable_rastrigin(3)
Best fitness: 11.713368947516107
Numer of demes: 3.32
Evaluation budget: 15126.6
-
Running attractive_sector(6)
Best fitness: 4.571526729634456e-05
Numer of demes: 6.04
Evaluation budget: 15818.0
-
Running step_elipsoid(7)
Best fitness: 0.558793602925852
Numer of demes: 6.96
Evaluation budget: 15420.2
-
Running bent_cigar(12)
Best fitness: 2.9314852825115802
Numer of demes: 2.84
Evaluation budget: 14988.0
-
Running asymetric_rastrigin(15)
Best fitness: 10.607915733871812
Numer of demes: 3.18
Evaluation budget: 15118.4
-
Running griewank_rosenbrock(19)
Best fitness: 5.631038627586743
Numer of demes: 3.4
Evaluation budget: 15132.6
-
Running katsuura(23)
Best fitness: 4.111493924286301
Numer of demes: 7.0
Evaluation budget: 16101.0
-
Running lunacek_rastrigin(24)
Best fitness: 48.83613418442732
Numer of demes: 2.8
Evaluation budget: 15000.0
-


#### Testing second attempt at deme hibernation

#### Testing with added local optimization demes

#### Testing Differential Evolution root deme

In [5]:
bounds_base = [(-5.0, 5.0)]
gsc = metaepoch_limit(35)
lvl_2_lsc = dont_stop()
lvl_1_generations = 1
lvl_2_generations = 8
pop_size = 40
dither = True
sigma0 = 2.0

dimensionality = 10

sprout_cond = SproutMechanism(NBC_Generator(2.0, 0.4), [NBC_FarEnough(2.0, 2), DemeLimit(1)], [LevelLimit(6)])

In [6]:
def setup_hms_with_de_for_bbob(function_problem, bounds):

    config = [
    DELevelConfig(
        generations=lvl_1_generations, 
        problem=function_problem, 
        bounds=bounds, 
        pop_size=pop_size,
        dither=dither,
        lsc=dont_stop()
        ),
    CMALevelConfig(
        generations=lvl_2_generations,
        problem=function_problem, 
        bounds=bounds,
        sigma0=sigma0,
        lsc=lvl_2_lsc
        )
    ]

    config = TreeConfig(config, gsc, sprout_cond)
    return DemeTree(config)

In [7]:
for name, problem in testbed.items():
    hms_tree = setup_hms_with_de_for_bbob(FunctionProblem(problem[0], maximize=False), bounds_base*dimensionality)
    print(f"Running {name}")
    while not hms_tree._gsc(hms_tree):
        hms_tree.metaepoch_count += 1
        hms_tree.run_metaepoch()
        if not hms_tree._gsc(hms_tree):
            hms_tree.run_sprout()

    f = lambda x: lvl_1_generations if x == 1 else lvl_2_generations
    print(f"Best fitness: {np.max(hms_tree.optima).fitness - problem[1]}")
    print(f"Numer of demes: {len(hms_tree.all_demes)}")
    print(f"Evaluation budget: {np.sum([np.sum([len(pop) for pop in deme.history])*(f(level)) for level, deme in hms_tree.all_demes])}")
    print('-')

Running separable_rastrigin(3)
Best fitness: 7.959667418927552
Numer of demes: 2
Evaluation budget: 11870
-
Running attractive_sector(6)
Best fitness: 0.0002819992172504726
Numer of demes: 2
Evaluation budget: 11870
-
Running step_elipsoid(7)
Best fitness: 8.24696062411013
Numer of demes: 3
Evaluation budget: 11880
-
Running bent_cigar(12)
Best fitness: 2.381627534981817
Numer of demes: 2
Evaluation budget: 11870
-
Running asymetric_rastrigin(15)
Best fitness: 12.934437471186243
Numer of demes: 2
Evaluation budget: 11870
-
Running griewank_rosenbrock(19)
Best fitness: 6.046868198591767
Numer of demes: 2
Evaluation budget: 11870
-
Running katsuura(23)
Best fitness: 3.7062478317622345
Numer of demes: 2
Evaluation budget: 11870
-
Running lunacek_rastrigin(24)
Best fitness: 68.09394418701471
Numer of demes: 2
Evaluation budget: 11870
-


#### Visualizing populations over time